In [1]:
#instalando o selenium
!pip install selenium

In [1]:
#imports
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from pprint import pprint
from selenium import webdriver
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.relative_locator import locate_with
import os

In [144]:
#link da página de certificações da RIAA filtrado de 01/01/2010 a 31/12/2019
URL = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar=&ti=&lab=&genre=&format=&date_option=release&from=2010-01-01&to=2019-12-31&award=&type=&category=&adv=SEARCH#search_section"

In [145]:
#usando o driver do selenium para abrir e maximizar a página em questão
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

driver.get(URL)
driver.maximize_window()

#criando um bot que carrega a página completa, clicando no botão "Load More" e depois exibe mais informações de cada certificação, clicando no botão "MORE DETAILS"
THERES_MORE = True

while THERES_MORE:

	WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='footer-content']")))

	try:
		driver.find_element_by_id('loadmore').click()
		time.sleep(1)
	except:
		THERES_MORE = False
        
like = driver.find_elements_by_link_text('MORE DETAILS')
for x in range(0,len(like)):
    if like[x].is_displayed():
        like[x].click()
        time.sleep(1)

#salvando o código fonte completo da página para possíveis consultas        
pageSource = driver.page_source
fileToWrite = open("page_source.html", "w")
fileToWrite.write(pageSource)
fileToWrite.close()
fileToRead = open("page_source.html", "r")
print(fileToRead.read())

<ipython-input-145-bce9b7da1a6a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_diamond = webdriver.Chrome(PATH_diamond)
<ipython-input-145-bce9b7da1a6a>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver_diamond.find_element_by_id('loadmore').click()
<ipython-input-145-bce9b7da1a6a>:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  like_diamond = driver_diamond.find_elements_by_link_text('MORE DETAILS')
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [300]:
#o tipo de certificação ("Gold", "Platinum", "Multi Platinum", "Diamond") é definido pela imagem exibida na faixa de cada certificação
#dessa forma, precisamos selecionar todas as imagens e filtrar pela URL das imagens que correspondem à certificações

#selecionando todas as imagens da página
img = driver.find_elements_by_tag_name('img')
links = []
for image in img:
    links.append(image.get_attribute('src'))
    #print(image.get_attribute('src'))
len(links)

<ipython-input-300-e2c64d48846c>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  img = driver_diamond.find_elements_by_tag_name('img')


48971

In [301]:
#selecionando as imagens que representam uma certificação
links_imgs = []
for i in links:
    if "https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/" and "_big.png" in i:
        links_imgs.append(i)

links_imgs.pop(0)
print(links_imgs)
len(links_imgs)

['https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/2_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/2_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/2_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/2_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_big.png', 'https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/0_b

13986

In [302]:
#a página teve bugs em 24 certificações, por esse motivo excluiremos essas informações das próximas tabelas

#selecionando a parcela do link que corresponde à certificação
award_links = []
for link in range(0,13986,2):
    award_links.append(links_imgs[link])
award_links = [i.replace("https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/", "") for i in award_links]
award_links = [i.replace("_big.png", "") for i in award_links]
award_links.pop(6297)
award_links.pop(3359)
award_links.pop(28)
award_links.pop(27)
award_links.pop(25)
award_links.pop(23)
award_links.pop(22)
award_links.pop(21)
award_links.pop(19)
award_links.pop(17)
award_links.pop(16)
award_links.pop(15)
award_links.pop(14)
award_links.pop(13)
award_links.pop(12)
award_links.pop(11)
award_links.pop(8)
award_links.pop(7)
award_links.pop(6)
award_links.pop(5)
award_links.pop(4)
award_links.pop(2)
award_links.pop(1)
print(award_links)
len(award_links)

['2', '2', '1', '1', '1', '5', '14', '4', '10', '1', '1', '0', '0', '0', '1', '1', '2', '0', '1', '0', '5', 'la_0', '0', '1', '14', '0', '4', 'la_3', '0', '10', 'la_80', '0', '3', '3', '0', '1', '3', '0', '0', '1', '0', '0', '1', '5', '4', '1', '0', '7', '0', '0', '0', '3', '0', '0', '0', '5', '3', '3', '1', '1', '0', '2', '1', '0', '0', '3', '5', '4', '6', '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '5', '0', '1', '0', '1', '9', '1', '3', '1', '2', '3', '1', '2', '0', '0', '1', '0', '1', '1', '3', '2', '1', '1', '2', '1', '3', '1', '1', '3', '0', '0', '0', '0', '3', '0', '0', '0', '2', '2', '0', '1', '7', 'la_0', '0', '0', '2', '0', '1', '1', '3', '2', '5', '2', '2', '5', 'la_8', 'la_1', 'la_2', 'la_8', 'la_2', 'la_1', 'la_4', 'la_0', '0', '9', 'la_1', '1', '0', 'la_26', 'la_3', 'la_4', '1', '0', '0', '0', 'la_7', 'la_9', 'la_3', 'la_4', 'la_31', 'la_5', 'la_10', 'la_4', '0', 'la_12', 'la_6', '0', 'la_15', '2', '1', '0', '7', '5', '0', '1', '2', '1', '1', '1', '1', '1', '0', '0'

6970

In [303]:
#retirando a especificação de certificação latina "la_" na string
new_award_links = []
for i in award_links:
    if "la_" in i:
        i = i.replace("la_", "")
        new_award_links.append(i)
    else:
        new_award_links.append(i)
        
print(new_award_links)
len(new_award_links)

['2', '2', '1', '1', '1', '5', '14', '4', '10', '1', '1', '0', '0', '0', '1', '1', '2', '0', '1', '0', '5', '0', '0', '1', '14', '0', '4', '3', '0', '10', '80', '0', '3', '3', '0', '1', '3', '0', '0', '1', '0', '0', '1', '5', '4', '1', '0', '7', '0', '0', '0', '3', '0', '0', '0', '5', '3', '3', '1', '1', '0', '2', '1', '0', '0', '3', '5', '4', '6', '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '5', '0', '1', '0', '1', '9', '1', '3', '1', '2', '3', '1', '2', '0', '0', '1', '0', '1', '1', '3', '2', '1', '1', '2', '1', '3', '1', '1', '3', '0', '0', '0', '0', '3', '0', '0', '0', '2', '2', '0', '1', '7', '0', '0', '0', '2', '0', '1', '1', '3', '2', '5', '2', '2', '5', '8', '1', '2', '8', '2', '1', '4', '0', '0', '9', '1', '1', '0', '26', '3', '4', '1', '0', '0', '0', '7', '9', '3', '4', '31', '5', '10', '4', '0', '12', '6', '0', '15', '2', '1', '0', '7', '5', '0', '1', '2', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '5', '7', '3', '3', '0', '0',

6970

In [304]:
#baseado no valor proveninente do link especificaremos cada uma das certificações
#0 = gold
#1 = platinum
#de 2 a 9 = multi platinum
#10 ou mais = diamond
awards = []
for i in new_award_links:
    if int(i) == 0:
        awards.append("Gold")
    if int(i) == 1:
        awards.append("Platinum")
    if 2 <= int(i) < 10:
        awards.append("Multi platinum")
    if int(i) >= 10 :
        awards.append("Diamond")
        
print(awards)   
len(awards)

['Multi platinum', 'Multi platinum', 'Platinum', 'Platinum', 'Platinum', 'Multi platinum', 'Diamond', 'Multi platinum', 'Diamond', 'Platinum', 'Platinum', 'Gold', 'Gold', 'Gold', 'Platinum', 'Platinum', 'Multi platinum', 'Gold', 'Platinum', 'Gold', 'Multi platinum', 'Gold', 'Gold', 'Platinum', 'Diamond', 'Gold', 'Multi platinum', 'Multi platinum', 'Gold', 'Diamond', 'Diamond', 'Gold', 'Multi platinum', 'Multi platinum', 'Gold', 'Platinum', 'Multi platinum', 'Gold', 'Gold', 'Platinum', 'Gold', 'Gold', 'Platinum', 'Multi platinum', 'Multi platinum', 'Platinum', 'Gold', 'Multi platinum', 'Gold', 'Gold', 'Gold', 'Multi platinum', 'Gold', 'Gold', 'Gold', 'Multi platinum', 'Multi platinum', 'Multi platinum', 'Platinum', 'Platinum', 'Gold', 'Multi platinum', 'Platinum', 'Gold', 'Gold', 'Multi platinum', 'Multi platinum', 'Multi platinum', 'Multi platinum', 'Gold', 'Gold', 'Platinum', 'Gold', 'Gold', 'Platinum', 'Gold', 'Gold', 'Platinum', 'Platinum', 'Multi platinum', 'Gold', 'Platinum', 'Gol

6970

In [322]:
awards.pop(6360)
len(awards)

6969

In [293]:
#selecionando todos os nomes de artistas
artist = driver.find_elements_by_class_name("artists_cell")
x1 = 0
artista = []
for artist in artist:
    artista.append(artist.text)
    x1 = x1+1
print(artista[0])
print(x1)

<ipython-input-293-5190be8dbf99>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  artist_diamond = driver_diamond.find_elements_by_class_name("artists_cell")


NIKO MOON
6993


In [294]:
artista.pop(6297)
artista.pop(3359)
artista.pop(28)
artista.pop(27)
artista.pop(25)
artista.pop(23)
artista.pop(22)
artista.pop(21)
artista.pop(19)
artista.pop(17)
artista.pop(16)
artista.pop(15)
artista.pop(14)
artista.pop(13)
artista.pop(12)
artista.pop(11)
artista.pop(8)
artista.pop(7)
artista.pop(6)
artista.pop(5)
artista.pop(4)
artista.pop(2)
artista.pop(1)

print(artista)
len(artista)

['NIKO MOON', 'BROCKHAMPTON', 'HOUNDMOUTH', 'CASTING CROWNS', 'YOUNG M.A', 'ARIZONA ZERVAS', 'THE CHAINSMOKERS', 'LEE BRICE', 'NICKI MINAJ', 'HOUNDMOUTH', 'CASTING CROWNS', 'IRATION', 'MATT MAHER', 'TORY LANEZ', 'TORY LANEZ', 'TORY LANEZ', 'TORY LANEZ', 'JACKSEPTICEYE & THE GREGORY BROTHERS', 'YOUNG M.A', 'ALL THAT REMAINS', 'ARIZONA ZERVAS', 'LA BANDA GORDA', 'NATHANIEL RATELIFF AND THE NIGHT SWEATS', 'STARSET', 'THE CHAINSMOKERS', '$TUPID YOUNG', 'LEE BRICE', 'ZION & LENNOX', '22GZ', 'NICKI MINAJ', 'NIO GARCIA, CASPER MAGICO, BAD BUNNY, OZUNA, DARELL & NICKY JAM', 'AFRO B', 'AVICII', 'NOAH CYRUS', 'JUICE WRLD', 'JUICE WRLD', 'JUICE WRLD', 'JUICE WRLD', 'PARTYNEXTDOOR', 'PARTYNEXTDOOR', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'RIHANNA', 'ZACH BRYAN', 'DJ SNAKE', 'DJ SNAKE & ALUNAGEORGE', 'DJ SNAKE, BIPOLAR SUNSHINE', 'DJ SNAKE, J BALVIN, TYGA', 'STILL WOOZY', 'STILL WOOZY', '2 CHAINZ

6970

In [324]:
artista.pop(6360)
len(artista)

6969

In [295]:
#selecionando maiores informações da faixa da certificação
infos = driver.find_elements_by_class_name("others_cell")
y1 = 0
for info in infos:
    #print(info.text)
    y1 = y1+1
print(y1)

<ipython-input-295-3aa56c9aeca1>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  infos_diamond = driver_diamond.find_elements_by_class_name("others_cell")


27972


In [296]:
#retirando das informações os títulos das faixas
titles = []
for title in range(0,27972,4):
    titles.append(infos[title].text)
titles.pop(6297)
titles.pop(3359)
titles.pop(28)
titles.pop(27)
titles.pop(25)
titles.pop(23)
titles.pop(22)
titles.pop(21)
titles.pop(19)
titles.pop(17)
titles.pop(16)
titles.pop(15)
titles.pop(14)
titles.pop(13)
titles.pop(12)
titles.pop(11)
titles.pop(8)
titles.pop(7)
titles.pop(6)
titles.pop(5)
titles.pop(4)
titles.pop(2)
titles.pop(1)
print(titles)
len(titles)

['GOOD TIME', 'SUGAR', 'SEDONA', 'NOBODY (FEAT. MATTHEW WEST)', 'BIG', 'ROXANNE', 'CLOSER', 'HARD TO LOVE', 'SUPER BASS', 'SEDONA', 'NOBODY (FEAT. MATTHEW WEST)', 'FALLING', 'ALL THE PEOPLE SAID AMEN', 'CHIXTAPE 5', 'THE TAKE (FEAT. CHRIS BROWN)', 'JERRY SPRUNGER (FEAT T. PAIN)', 'TALK TO ME (FEAT. RICH THE KID)', 'ALL THE WAY (I BELIEVE IN STEVE)', 'BIG', 'WHAT IF I WAS NOTHING', 'ROXANNE', 'MI MUJER ME GOBIERNA', 'I NEED NEVER GET OLD', 'MY DEMONS', 'CLOSER', 'MANDO (FEAT. MOZZY)', 'HARD TO LOVE', 'PIERDO LA CABEZA', 'SUBURBAN, PT. 2', 'SUPER BASS', 'TE BOTE (REMIX)', 'DROGBA (JOANNA)', 'LEVELS', 'JULY', '10 FEET', 'END OF THE ROAD', 'WASTED (FEAT. LIL UZI VERT)', 'RIDER', 'PARTYNEXTDOOR', 'WUS GOOD / CURIOUS', 'WOO', 'JAMES JOINT', 'YEAH, I SAID IT', 'LOVE ON THE BRAIN', 'FOURFIVESECONDS', 'CONSIDERATION', 'CLOSE TO YOU', 'DIAMONDS', 'LOVE THE WAY YOU LIE (PART II FT. EMINEM)', 'HIGHER', 'SAME OL’ MISTAKES', 'BITCH BETTER HAVE MY MONEY', 'SKIN', 'S&M REMIX', 'HEADING SOUTH', 'LET ME

6970

In [326]:
titles.pop(6360)
len(titles)

6969

In [297]:
#retirando das informações as datas de certificação
certifdate = []
for date in range(1,27972,4):
    certifdate.append(infos[date].text)
certifdate.pop(6297)
certifdate.pop(3359)
certifdate.pop(28)
certifdate.pop(27)
certifdate.pop(25)
certifdate.pop(23)
certifdate.pop(22)
certifdate.pop(21)
certifdate.pop(19)
certifdate.pop(17)
certifdate.pop(16)
certifdate.pop(15)
certifdate.pop(14)
certifdate.pop(13)
certifdate.pop(12)
certifdate.pop(11)
certifdate.pop(8)
certifdate.pop(7)
certifdate.pop(6)
certifdate.pop(5)
certifdate.pop(4)
certifdate.pop(2)
certifdate.pop(1)
print(certifdate)
len(certifdate)

['November 22, 2021', 'November 19, 2021', 'November 17, 2021', 'November 16, 2021', 'November 12, 2021', 'November 11, 2021', 'November 11, 2021', 'November 10, 2021', 'November 9, 2021', 'November 17, 2021', 'November 16, 2021', 'November 16, 2021', 'November 16, 2021', 'November 15, 2021', 'November 15, 2021', 'November 15, 2021', 'November 15, 2021', 'November 12, 2021', 'November 12, 2021', 'November 11, 2021', 'November 11, 2021', 'November 11, 2021', 'November 11, 2021', 'November 11, 2021', 'November 11, 2021', 'November 10, 2021', 'November 10, 2021', 'November 10, 2021', 'November 9, 2021', 'November 9, 2021', 'November 9, 2021', 'November 8, 2021', 'November 5, 2021', 'November 5, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4, 2021', 'November 4,

6970

In [327]:
certifdate.pop(6360)
len(certifdate)

6969

In [298]:
#retirando das informações as gravadoras de cada faixa
label = []
for label in range(2,27972,4):
    label.append(infos[label].text)
label.pop(6297)
label.pop(3359)
label.pop(28)
label.pop(27)
label.pop(25)
label.pop(23)
label.pop(22)
label.pop(21)
label.pop(19)
label.pop(17)
label.pop(16)
label.pop(15)
label.pop(14)
label.pop(13)
label.pop(12)
label.pop(11)
label.pop(8)
label.pop(7)
label.pop(6)
label.pop(5)
label.pop(4)
label.pop(2)
label.pop(1)
print(label)
len(label)

['RCA NASHVILLE', 'QUESTION EVERYTHING / RCA RECORDS', 'ROUGH TRADE', 'BEACH STREET RECORDS / REUNION RECORDS', 'M.A MUSIC / 3D', 'COLUMBIA', 'COLUMBIA', 'CURB RECORDS', 'CASH MONEY / REPUBLIC RECORDS', 'ROUGH TRADE', 'BEACH STREET RECORDS / REUNION RECORDS', 'MRI ENTERTAINMENT', 'ESSENTIAL RECORDS / PROVIDENT LABEL GROUP LLC', 'INTERSCOPE', 'INTERSCOPE', 'INTERSCOPE', 'INTERSCOPE', 'GREGORY RESIDENCE', 'M.A MUSIC / 3D', 'CRAFT RECORDINGS', 'COLUMBIA', 'LA OREJA MEDIA GROUP', 'FANTASY/STAX', 'FEARLESS', 'COLUMBIA', 'AFFICIALS / EMPIRE', 'CURB RECORDS', 'BABY RECORDS INC / GLAD EMPIRE', 'ATLANTIC', 'CASH MONEY / REPUBLIC RECORDS', 'FLOW LA MOVIE INC / GLAD EMPIRE', 'MARATHON ARTISTS', 'AVICII MUSIC AB / UNIVERSAL MUSIC AB', 'RECORDS LLC / COLUMBIA', 'GRADE A / INTERSCOPE RECORDS', 'GRADE A / INTERSCOPE RECORDS', 'GRADE A / INTERSCOPE RECORDS', 'GRADE A / INTERSCOPE RECORDS', 'OVO SOUND / WARNER RECORDS', 'OVO SOUND / WARNER RECORDS', 'ROC NATION', 'ROC NATION', 'ROC NATION', 'DEF JAM', 

6970

In [328]:
label.pop(6360)
len(label)

6969

In [299]:
#retirando das informações o formato da faixa (single, album, video longform, etc)
format = []
for formato in range(3,27972,4):
    format.append(infos[formato].text)
format.pop(6297)
format.pop(3359)
format.pop(28)
format.pop(27)
format.pop(25)
format.pop(23)
format.pop(22)
format.pop(21)
format.pop(19)
format.pop(17)
format.pop(16)
format.pop(15)
format.pop(14)
format.pop(13)
format.pop(12)
format.pop(11)
format.pop(8)
format.pop(7)
format.pop(6)
format.pop(5)
format.pop(4)
format.pop(2)
format.pop(1)
format = [i.replace("\nMORE DETAILS", "") for i in format]
print(format)
len(format)

['SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'ALBUM', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'ALBUM', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'ALBUM', 'SINGLE', 'S

6970

In [329]:
format.pop(6360)
len(format)

6969

In [282]:
#selecionando todas as datas de lançamento (de jan/2010 a dez/2019)
release = driver.find_elements_by_class_name("upper_style")
a1 = 0
for release in release:
    #print(release.text)
    a1 = a1+1
print(a1)

<ipython-input-282-c8920f7c5395>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  release_diamond = driver_diamond.find_elements_by_class_name("upper_style")


6969


In [284]:
for release in release:
    print(release.text)
len(release)

JULY 19, 2019
AUGUST 23, 2019
JANUARY 20, 2015
OCTOBER 12, 2018
JUNE 28, 2019
OCTOBER 10, 2019
JULY 29, 2016
APRIL 17, 2012
JUNE 21, 2011
JANUARY 20, 2015
OCTOBER 12, 2018
MARCH 16, 2010
APRIL 19, 2013
NOVEMBER 15, 2019
NOVEMBER 15, 2019
NOVEMBER 8, 2019
JUNE 21, 2018
JULY 22, 2016
JUNE 28, 2019
NOVEMBER 6, 2012
OCTOBER 10, 2019
AUGUST 12, 2016
AUGUST 21, 2015
AUGUST 1, 2013
JULY 29, 2016
NOVEMBER 17, 2017
APRIL 17, 2012
NOVEMBER 4, 2014
NOVEMBER 29, 2019
JUNE 21, 2011
APRIL 13, 2018
FEBRUARY 16, 2018
NOVEMBER 21, 2011
JULY 31, 2019
MARCH 8, 2019
MAY 23, 2018
JULY 10, 2018
MARCH 8, 2019
JULY 1, 2013
MAY 12, 2013
JANUARY 28, 2016
JANUARY 28, 2016
JANUARY 28, 2016
FEBRUARY 5, 2016
JANUARY 24, 2015
JANUARY 28, 2016
JANUARY 28, 2016
NOVEMBER 16, 2012
JUNE 21, 2011
JANUARY 28, 2016
JANUARY 28, 2016
MARCH 26, 2015
NOVEMBER 12, 2010
APRIL 11, 2011
SEPTEMBER 30, 2019
AUGUST 5, 2016
NOVEMBER 24, 2014
OCTOBER 16, 2015
JUNE 14, 2019
NOVEMBER 5, 2017
DECEMBER 7, 2018
JULY 22, 2015
JUNE 4, 2013
NOV

MAY 14, 2017
AUGUST 10, 2018
NOVEMBER 9, 2018
NOVEMBER 9, 2018
NOVEMBER 22, 2019
OCTOBER 2, 2015
NOVEMBER 22, 2019
JULY 12, 2019
SEPTEMBER 28, 2016
JUNE 13, 2019
JUNE 11, 2019
JANUARY 25, 2011
JUNE 12, 2019
APRIL 24, 2019
DECEMBER 13, 2019
AUGUST 12, 2014
FEBRUARY 1, 2017
JULY 26, 2011
NOVEMBER 3, 2015
OCTOBER 22, 2013
OCTOBER 25, 2019
JUNE 8, 2018
MARCH 9, 2018
OCTOBER 11, 2019
APRIL 5, 2019
OCTOBER 27, 2017
JUNE 20, 2019
DECEMBER 8, 2017
JUNE 22, 2010
JUNE 22, 2010
JUNE 21, 2019
NOVEMBER 16, 2018
APRIL 25, 2019
SEPTEMBER 18, 2012
MARCH 11, 2016
JUNE 3, 2016
JUNE 17, 2016
FEBRUARY 27, 2015
FEBRUARY 27, 2015
FEBRUARY 27, 2015
JULY 12, 2016
JULY 12, 2019
NOVEMBER 8, 2019
MARCH 25, 2016
MAY 18, 2018
DECEMBER 15, 2017
NOVEMBER 23, 2018
MARCH 27, 2012
JANUARY 9, 2012
SEPTEMBER 13, 2019
JANUARY 25, 2019
JUNE 21, 2011
JANUARY 17, 2019
JUNE 7, 2019
MAY 10, 2019
DECEMBER 11, 2012
JULY 20, 2010
DECEMBER 11, 2012
OCTOBER 5, 2010
OCTOBER 1, 2012
OCTOBER 7, 2016
JANUARY 20, 2017
JUNE 1, 2018
JUNE 

JANUARY 13, 2017
JUNE 24, 2016
APRIL 30, 2019
JULY 26, 2019
FEBRUARY 5, 2016
NOVEMBER 27, 2019
JUNE 16, 2017
FEBRUARY 12, 2016
SEPTEMBER 7, 2018
DECEMBER 16, 2013
MAY 22, 2015
JULY 13, 2018
MARCH 31, 2017
SEPTEMBER 5, 2017
MARCH 22, 2019
SEPTEMBER 22, 2017
NOVEMBER 22, 2010
JUNE 29, 2018
FEBRUARY 28, 2018
JUNE 21, 2013
NOVEMBER 30, 2018
DECEMBER 13, 2019
JULY 12, 2019
AUGUST 3, 2018
NOVEMBER 22, 2019
JULY 18, 2019
JULY 13, 2017
OCTOBER 12, 2017
SEPTEMBER 30, 2014
SEPTEMBER 25, 2014
NOVEMBER 10, 2014
MARCH 30, 2018
NOVEMBER 10, 2014
OCTOBER 24, 2014
OCTOBER 27, 2017
APRIL 17, 2018
AUGUST 3, 2018
NOVEMBER 22, 2018
NOVEMBER 22, 2019
MARCH 10, 2017
JUNE 9, 2018
AUGUST 9, 2019
NOVEMBER 3, 2017
MARCH 2, 2016
AUGUST 26, 2016
FEBRUARY 17, 2017
FEBRUARY 13, 2016
FEBRUARY 17, 2017
JUNE 30, 2017
MARCH 30, 2018
MAY 16, 2018
FEBRUARY 15, 2011
MAY 18, 2018
OCTOBER 21, 2016
OCTOBER 21, 2016
MAY 1, 2017
SEPTEMBER 22, 2017
JUNE 7, 2018
JULY 1, 2014
AUGUST 28, 2012
FEBRUARY 3, 2017
FEBRUARY 17, 2017
JUN

MARCH 28, 2018
MAY 25, 2018
JANUARY 15, 2013
AUGUST 28, 2019
OCTOBER 7, 2013
NOVEMBER 19, 2012
NOVEMBER 16, 2012
NOVEMBER 21, 2014
JUNE 21, 2011
JUNE 17, 2011
JUNE 17, 2011
JUNE 17, 2011
JULY 23, 2014
MARCH 27, 2012
NOVEMBER 21, 2014
FEBRUARY 8, 2019
OCTOBER 6, 2018
OCTOBER 26, 2018
MAY 25, 2018
JUNE 2, 2017
JUNE 8, 2018
FEBRUARY 7, 2012
JULY 5, 2018
FEBRUARY 14, 2018
NOVEMBER 6, 2018
MAY 17, 2019
SEPTEMBER 20, 2019
APRIL 27, 2018
OCTOBER 12, 2018
AUGUST 10, 2018
MARCH 14, 2011
MAY 24, 2011
SEPTEMBER 3, 2013
OCTOBER 29, 2019
OCTOBER 4, 2019
NOVEMBER 27, 2019
MAY 8, 2018
MAY 7, 2019
NOVEMBER 13, 2015
JULY 1, 2014
JULY 1, 2014
AUGUST 14, 2012
JANUARY 20, 2015
NOVEMBER 30, 2018
NOVEMBER 8, 2018
AUGUST 10, 2018
OCTOBER 6, 2017
JULY 24, 2019
MAY 14, 2017
NOVEMBER 19, 2019
NOVEMBER 23, 2016
NOVEMBER 12, 2019
MAY 14, 2017
NOVEMBER 1, 2019
NOVEMBER 9, 2018
NOVEMBER 22, 2019
MAY 29, 2019
NOVEMBER 22, 2019
AUGUST 10, 2018
AUGUST 6, 2018
NOVEMBER 9, 2018
NOVEMBER 9, 2018
SEPTEMBER 15, 2017
DECEMB

APRIL 27, 2018
APRIL 5, 2013
AUGUST 23, 2019
DECEMBER 20, 2011
FEBRUARY 21, 2012
FEBRUARY 2, 2015
MARCH 30, 2012
JULY 17, 2015
JANUARY 11, 2019
JUNE 8, 2012
AUGUST 7, 2017
SEPTEMBER 9, 2015
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
APRIL 20, 2018
DECEMBER 27, 2019
NOVEMBER 8, 2019
NOVEMBER 8, 2019
DECEMBER 13, 2019
FEBRUARY 14, 2019
MARCH 8, 2016
MARCH 17, 2017
MAY 12, 2017
OCTOBER 14, 2016
JANUARY 12, 2010
AUGUST 14, 2017
JULY 24, 2019
AUGUST 16, 2019
AUGUST 16, 2019
AUGUST 16, 2019
MAY 23, 2019
JUNE 15, 2018
DECEMBER 23, 2013
JUNE 15, 2012
NOVEMBER 13, 2015
NOVEMBER 1, 2011
NOVEMBER 1, 2011
JUNE 15, 2012
JUNE 15, 2012
JANUARY 29, 2013
DECEMBER 9, 2010
DECEMBER 23, 2013
OCTOBER 11, 2019
OCTOBER 12, 2012
NOVEMBER 13, 2015
NOVEMBER 1, 2011
MARCH 19, 2010
DECEMBER 23, 2013
NOVEMBER 26, 2010
FEBRUARY 16, 2018
APRIL 14, 2015
AUGUST 28, 2015
JANUARY 17, 2018
DECEMBER 17, 2013
JULY 20, 2018
AUGUST 30, 2019
APRIL 7,

...

AUGUST 9, 2011
AUGUST 7, 2015
OCTOBER 8, 2013
FEBRUARY 1, 2011
SEPTEMBER 21, 2010
SEPTEMBER 21, 2010
AUGUST 24, 2010
JULY 9, 2012
SEPTEMBER 21, 2010
SEPTEMBER 16, 2014
APRIL 24, 2012
SEPTEMBER 10, 2013
MAY 20, 2016
NOVEMBER 6, 2015
JULY 12, 2011
DECEMBER 23, 2011
AUGUST 24, 2015
JUNE 4, 2016
APRIL 7, 2017
SEPTEMBER 28, 2010
SEPTEMBER 9, 2013
MAY 6, 2017
APRIL 21, 2017
DECEMBER 24, 2015
OCTOBER 31, 2016
JUNE 23, 2017
JUNE 21, 2011
SEPTEMBER 17, 2013
NOVEMBER 20, 2015
SEPTEMBER 18, 2012
MARCH 11, 2016
MARCH 3, 2015
DECEMBER 6, 2012
MAY 22, 2012
DECEMBER 6, 2016
NOVEMBER 6, 2015
APRIL 28, 2015
AUGUST 12, 2014
AUGUST 5, 2014
SEPTEMBER 16, 2016
NOVEMBER 3, 2015
MAY 16, 2014
JUNE 12, 2012
JUNE 22, 2014
MAY 25, 2012
JUNE 17, 2014
SEPTEMBER 24, 2015
JULY 1, 2014
MARCH 26, 2012
SEPTEMBER 14, 2010
JUNE 26, 2012
MAY 27, 2011
FEBRUARY 16, 2017
OCTOBER 9, 2012
NOVEMBER 5, 2010
NOVEMBER 18, 2016
APRIL 29, 2016
MARCH 18, 2017
FEBRUARY 13, 2015
MAY 27, 2016
APRIL 12, 2011
JUNE 21, 2011
JULY 15, 2016
F

APRIL 13, 2010
JUNE 1, 2012
MAY 1, 2012
DECEMBER 17, 2013
JULY 23, 2014
JULY 15, 2014
JULY 8, 2014
APRIL 27, 2010
JUNE 30, 2015
OCTOBER 25, 2011
DECEMBER 4, 2012
SEPTEMBER 14, 2010
AUGUST 21, 2012
NOVEMBER 13, 2015
JUNE 18, 2013
SEPTEMBER 14, 2010
JUNE 1, 2015
MARCH 2, 2015
JANUARY 20, 2014
DECEMBER 13, 2011
APRIL 7, 2012
JUNE 25, 2013
OCTOBER 22, 2013
JUNE 12, 2012
APRIL 7, 2014
FEBRUARY 14, 2012
OCTOBER 27, 2014
MARCH 1, 2011
JANUARY 24, 2012
JULY 1, 2014
AUGUST 19, 2014
NOVEMBER 20, 2015
AUGUST 12, 2014
SEPTEMBER 30, 2012
JULY 22, 2014
DECEMBER 4, 2012
FEBRUARY 11, 2014
MAY 11, 2010
JUNE 11, 2013
AUGUST 27, 2010
JANUARY 3, 2012
APRIL 6, 2010
JUNE 15, 2010
JULY 17, 2015
AUGUST 28, 2012
JUNE 10, 2016
FEBRUARY 27, 2015
MARCH 29, 2011
MARCH 13, 2012
JULY 15, 2013
DECEMBER 4, 2012
JANUARY 25, 2011
JUNE 23, 2014
JANUARY 6, 2015
FEBRUARY 3, 2011
SEPTEMBER 14, 2010
OCTOBER 5, 2010
FEBRUARY 21, 2012
NOVEMBER 11, 2013
APRIL 11, 2013
AUGUST 14, 2015
FEBRUARY 25, 2014
MAY 13, 2016
JUNE 25, 2013

AUGUST 6, 2013
MAY 19, 2014
JUNE 24, 2014
FEBRUARY 28, 2012
OCTOBER 22, 2012
FEBRUARY 12, 2013
JANUARY 11, 2014
JULY 16, 2013
NOVEMBER 15, 2011
JUNE 28, 2011
MARCH 26, 2013
MARCH 19, 2013
AUGUST 12, 2011
MAY 19, 2014
JUNE 28, 2012
MAY 13, 2014
APRIL 16, 2013
MARCH 3, 2014
MAY 14, 2013
NOVEMBER 6, 2012
MARCH 18, 2014
MARCH 19, 2013
OCTOBER 12, 2010
APRIL 3, 2012
APRIL 3, 2012
AUGUST 22, 2013
JULY 3, 2012
OCTOBER 30, 2012
OCTOBER 7, 2011
JULY 30, 2013
OCTOBER 9, 2012
NOVEMBER 19, 2013
MARCH 20, 2012
APRIL 6, 2014
OCTOBER 25, 2010
JULY 18, 2013
MAY 2, 2014
OCTOBER 12, 2010
OCTOBER 25, 2010
OCTOBER 25, 2010
AUGUST 16, 2010
SEPTEMBER 16, 2013
MARCH 3, 2014
AUGUST 17, 2011
MARCH 3, 2014
NOVEMBER 9, 2010
MARCH 26, 2013
JULY 29, 2013
JUNE 10, 2013
JULY 9, 2013
APRIL 3, 2012
APRIL 3, 2012
JANUARY 10, 2012
DECEMBER 20, 2011
MARCH 26, 2013
SEPTEMBER 3, 2013
MARCH 18, 2014
OCTOBER 8, 2013
MAY 6, 2014
MAY 6, 2014
OCTOBER 21, 2013
DECEMBER 13, 2013
MAY 14, 2013
AUGUST 27, 2013
OCTOBER 22, 2013
NOVEM

OCTOBER 28, 2010
JANUARY 26, 2010
MARCH 23, 2010
MARCH 16, 2010
DECEMBER 20, 2010
MARCH 19, 2010
JANUARY 18, 2011
OCTOBER 25, 2010
AUGUST 8, 2010
MARCH 15, 2011
JUNE 17, 2010
NOVEMBER 1, 2011
MAY 3, 2011
FEBRUARY 15, 2011
OCTOBER 25, 2010
OCTOBER 5, 2010
OCTOBER 12, 2010
APRIL 27, 2010
OCTOBER 24, 2011
MAY 23, 2011
JANUARY 12, 2010
JUNE 28, 2011
DECEMBER 7, 2010
SEPTEMBER 17, 2010
NOVEMBER 22, 2010
SEPTEMBER 6, 2011
APRIL 12, 2011
JULY 14, 2011
JUNE 7, 2010
SEPTEMBER 13, 2011
FEBRUARY 1, 2011
MAY 4, 2010
AUGUST 30, 2011
JULY 28, 2011
JULY 6, 2010
JANUARY 25, 2011
MAY 2, 2011
FEBRUARY 22, 2011
AUGUST 12, 2011
MAY 11, 2010
JULY 20, 2010
FEBRUARY 2, 2010
JUNE 1, 2010
SEPTEMBER 28, 2010
JUNE 14, 2011
AUGUST 31, 2010
JUNE 18, 2010
MAY 18, 2010
JUNE 8, 2010
JUNE 28, 2011
APRIL 21, 2011
AUGUST 3, 2010
FEBRUARY 15, 2011
MAY 23, 2011
MAY 20, 2011
JULY 6, 2010
NOVEMBER 22, 2010
NOVEMBER 16, 2010
JUNE 14, 2011
JUNE 14, 2011
AUGUST 24, 2010
JULY 27, 2010
NOVEMBER 30, 2010
JANUARY 18, 2011
MAY 23, 

6969

In [306]:
releasedate = []
for date in range(0,6969):
    releasedate.append(release[date].text)
print(releasedate)
len(releasedate)

['JULY 19, 2019', 'AUGUST 23, 2019', 'JANUARY 20, 2015', 'OCTOBER 12, 2018', 'JUNE 28, 2019', 'OCTOBER 10, 2019', 'JULY 29, 2016', 'APRIL 17, 2012', 'JUNE 21, 2011', 'JANUARY 20, 2015', 'OCTOBER 12, 2018', 'MARCH 16, 2010', 'APRIL 19, 2013', 'NOVEMBER 15, 2019', 'NOVEMBER 15, 2019', 'NOVEMBER 8, 2019', 'JUNE 21, 2018', 'JULY 22, 2016', 'JUNE 28, 2019', 'NOVEMBER 6, 2012', 'OCTOBER 10, 2019', 'AUGUST 12, 2016', 'AUGUST 21, 2015', 'AUGUST 1, 2013', 'JULY 29, 2016', 'NOVEMBER 17, 2017', 'APRIL 17, 2012', 'NOVEMBER 4, 2014', 'NOVEMBER 29, 2019', 'JUNE 21, 2011', 'APRIL 13, 2018', 'FEBRUARY 16, 2018', 'NOVEMBER 21, 2011', 'JULY 31, 2019', 'MARCH 8, 2019', 'MAY 23, 2018', 'JULY 10, 2018', 'MARCH 8, 2019', 'JULY 1, 2013', 'MAY 12, 2013', 'JANUARY 28, 2016', 'JANUARY 28, 2016', 'JANUARY 28, 2016', 'FEBRUARY 5, 2016', 'JANUARY 24, 2015', 'JANUARY 28, 2016', 'JANUARY 28, 2016', 'NOVEMBER 16, 2012', 'JUNE 21, 2011', 'JANUARY 28, 2016', 'JANUARY 28, 2016', 'MARCH 26, 2015', 'NOVEMBER 12, 2010', 'A

6969

In [313]:
#selecionando todas as informações de unidades certificadas
certif_units = driver.find_elements_by_class_name("col-md-4")
c1 = 0
for certif in certif_units:
    #print(certif.text)
    c1 = c1+1
print(c1)

<ipython-input-313-8bb2e78fb07d>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  certif_units_diamond = driver_diamond.find_elements_by_class_name("col-md-4")


13968


In [315]:
units = []
for unit in certif_units:
    if "Million" in unit.text:
        units.append(unit.text)
print(units)
len(units)

['2 Million', '2 Million', '1 Million', '1 Million', '1 Million', '5 Million', '14 Million', '4 Million', '10 Million', '1 Million', '1 Million', '0.5 Million', '0.5 Million', '0.5 Million', '1 Million', '1 Million', '2 Million', '0.5 Million', '1 Million', '0.5 Million', '5 Million', '0.03 Million', '0.5 Million', '1 Million', '14 Million', '0.5 Million', '4 Million', '0.18 Million', '0.5 Million', '10 Million', '4.8 Million', '0.5 Million', '3 Million', '3 Million', '0.5 Million', '1 Million', '3 Million', '0.5 Million', '0.5 Million', '1 Million', '0.5 Million', '0.5 Million', '1 Million', '5 Million', '4 Million', '1 Million', '0.5 Million', '7 Million', '0.5 Million', '0.5 Million', '0.5 Million', '3 Million', '0.5 Million', '0.5 Million', '0.5 Million', '5 Million', '3 Million', '3 Million', '1 Million', '1 Million', '0.5 Million', '2 Million', '1 Million', '0.5 Million', '0.5 Million', '3 Million', '5 Million', '4 Million', '6 Million', '0.5 Million', '0.5 Million', '1 Million',

6969

In [331]:
#criando o dataframe com as listas
dados_certification = {"Certificação" : awards,
         "Artista(s)" : artista, 
         "Título" : titles,
         "Formato" : format,
         "Data de Lançamento" : releasedate,
         "Data de Certificação" : certifdate,
         "Gravadora(s)" : label,
         "Unidades Certificadas" : units
         }
df_certification = pd.DataFrame(dados_certification)
df_certification

,Certificação,Artista(s),Título,Formato,Data de Lançamento,Data de Certificação,Gravadora(s),Unidades Certificadas
0,Multi platinum,NIKO MOON,GOOD TIME,SINGLE,"JULY 19, 2019","November 22, 2021",RCA NASHVILLE,2 Million
1,Multi platinum,BROCKHAMPTON,SUGAR,SINGLE,"AUGUST 23, 2019","November 19, 2021",QUESTION EVERYTHING / RCA RECORDS,2 Million
2,Platinum,HOUNDMOUTH,SEDONA,SINGLE,"JANUARY 20, 2015","November 17, 2021",ROUGH TRADE,1 Million
3,Platinum,CASTING CROWNS,NOBODY (FEAT. MATTHEW WEST),SINGLE,"OCTOBER 12, 2018","November 16, 2021",BEACH STREET RECORDS / REUNION RECORDS,1 Million
4,Platinum,YOUNG M.A,BIG,SINGLE,"JUNE 28, 2019","November 12, 2021",M.A MUSIC / 3D,1 Million
...,...,...,...,...,...,...,...,...
6964,Gold,MONICA,STILL STANDING,ALBUM,"MARCH 23, 2010","April 21, 2010",J RECORDS,0.5 Million
6965,Platinum,LUDACRIS,HOW LOW,SINGLE,"MARCH 9, 2010","April 9, 2010",DEF JAM,1 Million
6966,Platinum,BANDA LOS RECODITOS,ANDO BIEN PEDO,ALBUM,"JANUARY 19, 2010","March 19, 2010",DISA,0.06 Million
6967,Platinum,SADE,SOLDIER OF LOVE,ALBUM,"FEBRUARY 9, 2010","March 15, 2010",COLUMBIA,1 Million


In [332]:
#salvando .csv com o dataframe
df_certification.to_csv('tabela-certification.csv')